# ランダムフォレスト

付近のwifiアクセスポイントをランダムフォレスト(機会学習)で学習し、登録されたどの位置に近いかを判定する

required sklearn, access-points

```
$ pip install access-points sklearn
```

もしくは

```
$ pip install -r requirements.txt
```

In [1]:
import os, pickle, io, csv, json

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from access_points import get_scanner

### アクセスポイント(データセット)の取得関数

In [2]:
def scan_access_points(label='default place', size=1):
    print('start scanning {} access points {} times.'.format(label, size))
    X = []
    y = []
    wifi_scanner = get_scanner()
    for index in range(int(size)):
        print('scan ' + str(index))
        access_points = wifi_scanner.get_access_points()
        X.append({' '.join([ap.ssid, ap.bssid]): ap.quality for ap in access_points })
        y.append(label)
    return X, y

データセットの中身

In [3]:
X, y = scan_access_points()
for Xi, yi in zip(X, y):
    print('{} access points = {}'.format(yi, Xi))

start scanning default place access points 1 times.
scan 0
default place access points = {'30F772C78D8E-2G 30:f7:72:c7:8d:90': 116, 'aterm-935798-g c0:25:a2:9e:66:f3': 38, 'aterm-935798-a c0:25:a2:9e:66:f2': 28, '30F772C78D8E-5G 30:f7:72:c7:8d:91': 122, 'aterm-935798-gw c2:25:a2:9e:66:f3': 38, 'aterm-935798-aw c2:25:a2:9e:66:f2': 28}


### モデルの定義

ランダムフォレストを定義。
DictVectorizerで、hash型のデータをいい感じに補完してもらう(ないkeyは0にするなど)

In [4]:
model = make_pipeline(
    DictVectorizer(sparse=False),
    RandomForestClassifier(n_estimators=500, class_weight='balanced')
)

### サンプリング(データ集め)

In [5]:
X = []
y = []

リビングに移動して下記を実行します。もしくはlabelを書き換えてください。

In [6]:
_X, _y = scan_access_points(label='living room', size=10)
X.extend(_X)
y.extend(_y)

start scanning living room access points 10 times.
scan 0
scan 1
scan 2
scan 3
scan 4
scan 5
scan 6
scan 7
scan 8
scan 9


寝室に移動して下記を実行します。もしくはlabelを書き換えてください。

In [7]:
_X, _y = scan_access_points(label='bedroom room', size=10)
X.extend(_X)
y.extend(_y)

start scanning bedroom room access points 10 times.
scan 0
scan 1
scan 2
scan 3
scan 4
scan 5
scan 6
scan 7
scan 8
scan 9


トイレに移動して下記を実行します。もしくはlabelを書き換えてください。

In [8]:
_X, _y = scan_access_points(label='toilet', size=10)
X.extend(_X)
y.extend(_y)

start scanning toilet access points 10 times.
scan 0
scan 1
scan 2
scan 3
scan 4
scan 5
scan 6
scan 7
scan 8
scan 9


キッチンに移動して下記を実行します。もしくはlabelを書き換えてください。

In [9]:
_X, _y = scan_access_points(label='kitchen', size=10)
X.extend(_X)
y.extend(_y)

start scanning kitchen access points 10 times.
scan 0
scan 1
scan 2
scan 3
scan 4
scan 5
scan 6
scan 7
scan 8
scan 9


### トレーニング実行

In [10]:
model.fit(X, y)

Pipeline(steps=[('dictvectorizer', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 登録されている場所

In [11]:
place_names = model.classes_
print(place_names)

['bedroom room' 'kitchen' 'living room' 'toilet']


### 場所の推論

登録されている場所に移動して実行します

In [12]:
X, _ = scan_access_points(size=1)
place_name = model.predict(X[0])[0]
print(place_name)

start scanning default place access points 1 times.
scan 0
living room


### 推論結果の割合

登録されている場所に移動して実行します

In [13]:
X, _ = scan_access_points(size=1)
percentages = {x: y for x, y in zip(place_names, model.predict_proba(X[0])[0])}
print(percentages)

start scanning default place access points 1 times.
scan 0
{'kitchen': 0.34999999999999998, 'toilet': 0.014, 'living room': 0.44400000000000001, 'bedroom room': 0.192}
